In [ ]:
import plotly.express as px
import json
from nnsight import LanguageModel
import json
import numpy as np

In [ ]:
model = LanguageModel("meta-llama/Llama-2-70b-hf")

with open('experimental_outputs/patching_results.json', 'r') as f:
    out = json.load(f)[-1]
false_prompt = out['false_prompt']
logit_diffs = out['logit_diffs']
n_toks = len(logit_diffs)
model_name = out['model']

# transpose logit_diffs
logit_diffs = [[logit_diffs[i][j] for i in range(0, len(logit_diffs))[::-1]] for j in range(len(logit_diffs[0]))]
probs = [[1 / (1 + np.exp(-logit)) for logit in layer] for layer in logit_diffs]

token_ids = model.tokenizer(false_prompt)['input_ids']
tokens = [
    model.tokenizer.decode([token_id]) + f" ({idx})" for idx, token_id in enumerate(token_ids)
]
tokens = tokens[-n_toks:]

fig = px.imshow(
    logit_diffs,
    x=tokens,
    labels=dict(x="Token", y="Layer"),
    color_continuous_scale='blues',
)
fig.show()